In [30]:
from __future__ import division
import numpy as np
import os
import pandas as pd
import sys
import scipy 
import scipy.stats 

In [10]:
def read_in_data(behav_data_f):
    """
    Read in the data
    """
    df = pd.read_csv(behav_data_f)
    df = df.loc[df['func_perc_fd'].notnull(), :]
    df = df.loc[df['FILE_ID']!='no_filename', :]
    df['AGE_YRS'] = np.floor(df['AGE_AT_SCAN'])

    return df


def split_half_outcome(df, motion_thresh, age_l, age_u, n, n_perms=100):
    
    """
    This function returns the R squared of how each parameter affects split-half reliability!
    It takes in a dataframe, motion threshold, an age upper limit(age_u) an age lower limit (age_l), sample size (n),
    and number of permutations (n_perms, currently hard coded at 100). This function essentially splits a data frame 
    into two matched samples (split_two_matched_samples.py), then creates mean roi-roi correlation matrices per sample 
    (make_group_corr_mat.py) and then calculates the R squared (calc_rsq.py) between the two samples'
    correlation matrices and returns all the permuation coefficients of determinations in a dataframe.
    """
    
    #set up data frame of average R squared to fill up later
    Rsq_list = []
    ICC_list = []
    ## set up age-motion corr dfs
    r_a_list=[]
    p_a_list=[]
    r_b_list=[]
    p_b_list=[]
    
    #Do this in each permutation
    for i in range(n_perms):
        #create two matched samples split on motion_thresh, age upper, age lower, and n
        df_A, df_B = split_two_matched_samples(df, motion_thresh, age_l, age_u, n)
        #make the matrix of all subjects roi-roi correlations, make the mean corr mat, and make covariance cor mat
        #do this for A and then B
        all_corr_mat_A, age_roi_corr_A = make_group_corr_mat(df_A)
        all_corr_mat_B, age_roi_corr_B = make_group_corr_mat(df_B)
        
        #calculate the R squared between the two matrices
        Rsq = calc_rsq(age_roi_corr_A, age_roi_corr_B)
        
        #calculate the ICC between the two matrices
        ICC = compute_icc(age_roi_corr_A, age_roi_corr_B)
        
        print ("Iteration " + str(i) + ": R^2 = " + str(Rsq) + ", ICC = " + str(ICC))
        
        #build up R squared output
        Rsq_list += [Rsq]
        ICC_list += [ICC]


        #check if age and motion are correlated
        age=df_A["AGE_AT_SCAN"]
        motion=df_A["func_perc_fd"]
        r_a,p_a=scipy.stats.pearsonr(age, motion) #returns r and p
    
        age=df_B["AGE_AT_SCAN"]
        motion=df_B["func_perc_fd"]
        r_b,p_b=scipy.stats.pearsonr(age, motion) #returns r and p

        #build up R and p output
        r_a_list += [r_a]
        p_a_list += [p_a]
        r_b_list += [r_b]
        p_b_list += [p_b]
    
    return np.array(Rsq_list), np.array(ICC_list), np.array(r_a_list), np.array(p_a_list),np.array(r_b_list), np.array(p_b_list)

def calc_rsq(av_corr_mat_A, av_corr_mat_B):
    """
    From wikipedia: https://en.wikipedia.org/wiki/Coefficient_of_determination
    
    Rsq = 1 - (SSres / SStot)
    
    SSres is calculated as the sum of square errors (where the error
    is the difference between x and y).
    
    SStot is calculated as the total sum of squares in y.
    """
    # Get the data we need
    inds = np.triu_indices_from(av_corr_mat_B, k=1)
    x = av_corr_mat_A[inds]
    y = av_corr_mat_B[inds]
    
    # Calculate the error/residuals
    res = y - x

    SSres = np.sum(res**2)
    
    # Sum up the total error in y
    y_var = y - np.mean(y)
    
    SStot = np.sum(y_var**2)
    
    # R squared
    Rsq = 1 - (SSres/SStot)
    
    return Rsq

def exclude_nan(x,y):
    """
    Exclude NaN values if either entry in a pair of vectors has NaN
    """
    idx = np.logical_not(np.logical_or(np.isnan(x), np.isnan(y)))
    x = x[idx]
    y = y[idx]
    n = len(x)
    return [x, y, n]

def compute_icc(av_corr_mat_A, av_corr_mat_B):
    """
    This function computes the inter-class correlation (ICC) of the
    two classes represented by the x and y numpy vectors.
    from: http://stats.stackexchange.com/questions/63368/intra-class-correlation-and-experimental-design
    and: Shrout, P. E., & Fleiss, J. L. (1979). Intraclass Correlations: Uses
    in Assessing Rater Reliability. Psychological Bulletin, 86(2), 420-428. http://rokwa.x-y.net/Shrout-Fleiss-ICC.pdf
    """

    inds = np.triu_indices_from(av_corr_mat_B, k=1)
    x = av_corr_mat_A[inds]
    y = av_corr_mat_B[inds]
    
    if all(x == y):
        return 1

    [x, y, n] = exclude_nan(x,y)

    ## Need at least 3 data points to compute this
    if n < 3:
        return np.nan

    Sx = sum(x); Sy = sum(y);
    Sxx = sum(x*x); Sxy = sum( (x+y)**2 )/2; Syy = sum(y*y)

    fact = ((Sx + Sy)**2)/(n*2)
    SS_tot = Sxx + Syy - fact
    SS_among = Sxy - fact
    SS_error = SS_tot - SS_among

    MS_error = SS_error/n
    MS_among = SS_among/(n-1)
    ICC = (MS_among - MS_error) / (MS_among + MS_error)

    return ICC

def make_group_corr_mat(df):
    """
    This function reads in each subject's aal roi time series files and creates roi-roi correlation matrices
    for each subject and then sums them all together. This creates  a 3d matrix of all subjects 
    roi-roi correlations. These correlations are Z scored. Then each roi-roi Z-scored correlation is correlated with age 
    across all subjects. The output of this function is aa 3d matrix of all subjects roi-roi correlations
    and a 2d matrix of r values for each roi-roi correlation with age.  
    """

    # for each subject do the following
    
    for i, (sub, f_id) in enumerate(df[['SUB_ID', 'FILE_ID']].values):
        
        #read each subjects aal roi time series files
        ts_df = pd.read_table('DATA/{}_rois_aal.1D'.format(f_id))

        #create a correlation matrix from the roi all time series files
        corr_mat_r = ts_df.corr()
        #the correlations need to be transformed to Fisher z, which is
        #equivalent to the arctanh function.
        corr_mat_z = np.arctanh(corr_mat_r)
        
        #for the first subject, add a correlation matrix of zeros that is the same dimensions as the aal roi-roi matrix
        if i == 0:
            all_corr_mat = np.zeros([corr_mat_z.shape[0], corr_mat_z.shape[1], len(df)])

        #now add the correlation matrix you just created for each subject to the all_corr_mat matrix (3D)
        all_corr_mat[:, :, i] = corr_mat_z

	##now correlate with age for each matrix
    age=df.loc[:, 'AGE_AT_SCAN']
    age_df=pd.DataFrame(age)
    age_df.index=[x for x in range(age_df.shape[0])]
    age_roi_corr =np.ones((116,116))
    for  r in range(all_corr_mat.shape[0]):
            for rn in range(all_corr_mat.shape[0]):
                if rn != r:
                    corr_dat=pd.DataFrame(all_corr_mat[r,rn,:])
                    corr_dat["age"]=age_df
                    age_roi_corr[r,rn]=corr_dat.corr()["age"][0]
    
    #create the mean correlation matrix (ignore nas - sometime there are some...)
    #av_corr_mat = np.nanmean(all_corr_mat, axis=2)
    #create the group covariance matrix (ignore nas - sometime there are some...)
    #var_corr_mat = np.nanvar(all_corr_mat, axis=2)
        
    return all_corr_mat, age_roi_corr

def split_two_matched_samples(df, motion_thresh, age_l, age_u, n):
    """
    This function takes in a data frame, thresholds it to only include
    participants whose percentage bad frames are less than motion_thresh
    and participants who are between the lower and upper age limits (inclusive),
    then returns two matched samples of size n. The samples are matched on
    age in years, autism diagnosis, gender and scanning site. This function also selectively samples the
    func_perc_fd
    Information about the motion measure is here:
    http://preprocessed-connectomes-project.org/quality-assessment-protocol/
    """
    
    # Start by removing all participants whose data is below a certain
    # motion threshold.
    df_samp_motion =  df.loc[df['func_perc_fd'] < motion_thresh, :]

    # Then remove participants who are younger (in years) than age_l and older
    # than age_u. Note that this means people who are age_l and age_u
    # (eg 6 and 10) will be included in the sample.
    df_samp = df_samp_motion.loc[(df_samp_motion['AGE_YRS']>=age_l)
                                    & (df_samp_motion['AGE_YRS']<=age_u), :]
                                    
    ##sort subjects based on motion
    sort_column_list = ['func_perc_fd']
    df_motion_sorted = df_samp.sort_values(by=sort_column_list)
    
    ##rank subjects by motion
    r=range(len(df_motion_sorted))
    r_df=pd.DataFrame(r)
    r_df.columns = ['rank']
    r_df['newcol'] = df_motion_sorted.index
    r_df.set_index('newcol', inplace=True)
    r_df.index.names = [None]
    df_motion_sorted_rank=pd.concat ([r_df,df_motion_sorted], axis=1)
    
    ##create bins of subjects in quartiles
    l=len(df_motion_sorted_rank)
    chunk=l/4
    chunk1=chunk
    chunk2=2*chunk
    chunk3=3*chunk
    chunk4=l
    
    first=df_motion_sorted_rank[df_motion_sorted_rank['rank']<=chunk1]
    second=df_motion_sorted_rank[(df_motion_sorted_rank['rank']>chunk1) & (df_motion_sorted_rank['rank']<=chunk2)]
    third=df_motion_sorted_rank[(df_motion_sorted_rank['rank']>chunk2) & (df_motion_sorted_rank['rank']<=chunk3)]
    fourth=df_motion_sorted_rank[df_motion_sorted_rank['rank']>=chunk3]
    
    ##take 2n/4 from each bin
    n_samp=(n*2)/4
    n_samp
    n_samp=int(n_samp)

    # Shuffle these remaining participants to ensure you get different sub
    # samples each time you run the code.
    first_rand = first.reindex(np.random.permutation(first.index))
    second_rand = second.reindex(np.random.permutation(second.index))
    third_rand = third.reindex(np.random.permutation(third.index))
    fourth_rand = fourth.reindex(np.random.permutation(fourth.index))

    # Only keep the top 2*n/4 participants.
    first_samp_2n = first_rand.iloc[:n_samp, :]
    second_samp_2n = second_rand.iloc[:n_samp, :]
    third_samp_2n = third_rand.iloc[:n_samp, :]
    fourth_samp_2n = fourth_rand.iloc[:n_samp, :]
    
    #append these together
    frames = [first_samp_2n, second_samp_2n, third_samp_2n,fourth_samp_2n]
    final_df = pd.concat(frames)

    # Sort these participants according to the sort columns of interest
    sort_column_list = ['DSM_IV_TR', 'DX_GROUP', 'SITE_ID', 'SEX', 'AGE_YRS']
    df_samp_2n_sorted = final_df.sort_values(by=sort_column_list)

    # Now put all even numbered participants in group A and all odd numbered
    # participants in group B.
    df_grp_A = df_samp_2n_sorted.iloc[::2, :]
    df_grp_B = df_samp_2n_sorted.iloc[1::2, :]

    # Boom! Return these two data frames
    return df_grp_A, df_grp_B



In [42]:
def abide_motion_wrapper(motion_thresh, age_l, age_u, n, n_perms=1000, overwrite=True):
    behav_data_f = '../../Phenotypic_V1_0b_preprocessed1.csv'
       
    f_name = 'RESULTS_age/rsq_{:03.0f}pct_{:03.0f}subs_{:02.0f}to{:02.0f}.csv'.format(motion_thresh, n, age_l, age_u)
    
    # By default this code will recreate files even if they already exist
    # (overwrite=True)
    # If you don't want to do this though, set overwrite to False and 
    # this step will skip over the analysis if the file already exists
    if not overwrite:
        # If the file exists then skip this loop
        if os.path.isfile(f_name):
            return
    
    df = read_in_data(behav_data_f)

    rsq_list, icc_list, r_a_list, p_a_list, r_b_list, p_b_list = split_half_outcome(df, motion_thresh, age_l, age_u, n, n_perms=n_perms)
    
    
    print ("R Squared list shape: " + str(rsq_list.shape))
    print ("ICC list shape: " + str(icc_list.shape))
    
    med_rsq = np.median(rsq_list)
    rsq_CI = np.percentile(rsq_list, 97.5) - np.percentile(rsq_list, 2.5)
    
    med_icc = np.median(icc_list)
    icc_CI = np.percentile(icc_list, 97.5) - np.percentile(icc_list, 2.5)

    med_r_a = np.median(r_a_list)
    med_p_a = np.median(p_a_list)
    med_r_b = np.median(r_b_list)
    med_p_b = np.median(p_b_list)


    columns = [ 'motion_thresh', 'age_l', 'age_u', 'n', 'med_rsq', 'CI_95', 'med_icc', 'CI_95_icc','med_age_motion_r_a','med_age_motion_p_a','med_age_motion_r_b','med_age_motion_p_b' ]
    results_df = pd.DataFrame(np.array([[motion_thresh, age_l, age_u, n, med_rsq, rsq_CI, med_icc, icc_CI, med_r_a, med_p_a, med_r_b, med_p_b ]]), 
                                  columns=columns)


    results_df.to_csv(f_name)


In [13]:
behav_data_f = '../../Phenotypic_V1_0b_preprocessed1.csv'
df = pd.read_csv(behav_data_f)
df = df.loc[df['func_perc_fd'].notnull(), :]
df = df.loc[df['FILE_ID']!='no_filename', :]
df['AGE_YRS'] = np.floor(df['AGE_AT_SCAN'])

In [17]:
df_A, df_B = split_two_matched_samples(df, 50, 6, 18, 100)

In [18]:
all_corr_mat_A, age_roi_corr_A = make_group_corr_mat(df_A)
all_corr_mat_B, age_roi_corr_B = make_group_corr_mat(df_B)

/om/user/jlnrd/py27/lib/python2.7/site-packages/ipykernel/__main__.py:169: RuntimeWarning: divide by zero encountered in arctanh


In [24]:
age_roi_corr_A.shape

(116, 116)

In [25]:
def split_half_outcome(df, motion_thresh, age_l, age_u, n, n_perms=100):
    
    """
    This function returns the R squared of how each parameter affects split-half reliability!
    It takes in a dataframe, motion threshold, an age upper limit(age_u) an age lower limit (age_l), sample size (n),
    and number of permutations (n_perms, currently hard coded at 100). This function essentially splits a data frame 
    into two matched samples (split_two_matched_samples.py), then creates mean roi-roi correlation matrices per sample 
    (make_group_corr_mat.py) and then calculates the R squared (calc_rsq.py) between the two samples'
    correlation matrices and returns all the permuation coefficients of determinations in a dataframe.
    """
    
    #set up data frame of average R squared to fill up later
    Rsq_list = []
    ICC_list = []
    ## set up age-motion corr dfs
    r_a_list=[]
    p_a_list=[]
    r_b_list=[]
    p_b_list=[]
    
    #Do this in each permutation
    for i in range(n_perms):
        #create two matched samples split on motion_thresh, age upper, age lower, and n
        df_A, df_B = split_two_matched_samples(df, motion_thresh, age_l, age_u, n)
        #make the matrix of all subjects roi-roi correlations, make the mean corr mat, and make covariance cor mat
        #do this for A and then B
        all_corr_mat_A, age_roi_corr_A = make_group_corr_mat(df_A)
        all_corr_mat_B, age_roi_corr_B = make_group_corr_mat(df_B)
        
        #calculate the R squared between the two matrices
        Rsq = calc_rsq(age_roi_corr_A, age_roi_corr_B)
        
        #calculate the ICC between the two matrices
        ICC = compute_icc(age_roi_corr_A, age_roi_corr_B)
        
        print ("Iteration " + str(i) + ": R^2 = " + str(Rsq) + ", ICC = " + str(ICC))
        
        #build up R squared output
        Rsq_list += [Rsq]
        ICC_list += [ICC]


        #check if age and motion are correlated
        age=df_A["AGE_AT_SCAN"]
        motion=df_A["func_perc_fd"]
        r_a,p_a=scipy.stats.pearsonr(age, motion) #returns r and p
    
        age=df_B["AGE_AT_SCAN"]
        motion=df_B["func_perc_fd"]
        r_b,p_b=scipy.stats.pearsonr(age, motion) #returns r and p

        #build up R and p output
        r_a_list += [r_a]
        p_a_list += [p_a]
        r_b_list += [r_b]
        p_b_list += [p_b]
    
    return np.array(Rsq_list), np.array(ICC_list), np.array(r_a_list), np.array(p_a_list),np.array(r_b_list), np.array(p_b_list)


In [31]:
rsq_list, icc_list,r_a_list,p_a_list,r_b_list,p_b_list = split_half_outcome(df, 50, 6, 18, 20, n_perms=10)

/om/user/jlnrd/py27/lib/python2.7/site-packages/ipykernel/__main__.py:169: RuntimeWarning: divide by zero encountered in arctanh


Iteration 0: R^2 = -0.943162920423, ICC = 0.0999202922312
Iteration 1: R^2 = -0.988460409123, ICC = 0.0835285726864
Iteration 2: R^2 = -1.70373081319, ICC = -0.154763006887
Iteration 3: R^2 = -1.08828607964, ICC = -0.0351962312366
Iteration 4: R^2 = -0.823892426984, ICC = 0.0693012890717
Iteration 5: R^2 = -0.688549487013, ICC = 0.137746427721
Iteration 6: R^2 = -2.70186857605, ICC = -0.298752452624
Iteration 7: R^2 = -1.15298939725, ICC = 0.040889359289
Iteration 8: R^2 = -1.23753538521, ICC = -0.103724378975
Iteration 9: R^2 = -1.15296718905, ICC = 0.0689110991836


In [32]:
p_a_list

array([ 0.16745234,  0.81544966,  0.23218532,  0.85965136,  0.33097956,
        0.0833455 ,  0.1659351 ,  0.39645775,  0.87851164,  0.31652746])

In [34]:
rsq_list

array([-0.94316292, -0.98846041, -1.70373081, -1.08828608, -0.82389243,
       -0.68854949, -2.70186858, -1.1529894 , -1.23753539, -1.15296719])

In [41]:
med_r_a = np.median(r_a_list)

In [40]:
med_r_a

-0.17583394362001187